In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
# !pip install fuzzywuzzy
# !pip install scikit-surprise

In [4]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from surprise import Reader, Dataset
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import SlopeOne
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import GridSearchCV


In [5]:
from datetime import datetime
import os
import random
import gc

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [6]:
movies = pd.read_csv('movies.csv')
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [8]:
movies['newId'] = range(1, movies['movieId'].nunique() + 1)

In [9]:
movies.head(3)

,movieId,title,genres,newId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
1,2,Jumanji (1995),Adventure|Children|Fantasy,2
2,3,Grumpier Old Men (1995),Comedy|Romance,3


In [10]:
movie_ratings = pd.read_csv('ratings.csv')
movie_ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [11]:
movie_ratings['timestamp'] = movie_ratings['timestamp'].apply(lambda x : datetime.utcfromtimestamp(x).strftime("%Y-%m-%d"))

In [12]:
movie_ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02
1,1,29,3.5,2005-04-02
2,1,32,3.5,2005-04-02
3,1,47,3.5,2005-04-02
4,1,50,3.5,2005-04-02


In [13]:
movie_ratings = movie_ratings.merge(movies, how = "left", on = "movieId")

In [14]:
movie_ratings.head()

,userId,movieId,rating,timestamp,title,genres,newId
0,1,2,3.5,2005-04-02,Jumanji (1995),Adventure|Children|Fantasy,2
1,1,29,3.5,2005-04-02,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,29
2,1,32,3.5,2005-04-02,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,32
3,1,47,3.5,2005-04-02,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,47
4,1,50,3.5,2005-04-02,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,50


In [15]:
movie_ratings.rename(columns = {"timestamp" : "date"}, inplace=True)

In [16]:
movie_ratings["movieId"] = movie_ratings['newId']
movies['movieId'] = movies['newId']

In [17]:
movie_ratings.drop(['newId'],axis=1, inplace=True)
movies.drop(['newId'], axis=1, inplace=True)

In [19]:
movie_ratings.sort_values(by="date", inplace=True)
movie_ratings.reset_index(drop=True, inplace=True)
# movies.sort_values(by="date", inplace=True)

In [20]:
print(movie_ratings.shape)
print(movie_ratings.columns)

(20000263, 6)
Index(['userId', 'movieId', 'rating', 'date', 'title', 'genres'], dtype='object')


Data Cleaning
